### Import libraries

In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

### Get the API keys from OS

In [22]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass()



In [23]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

### Initialize the model.

In [24]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

### Create the knowledge base.

In [33]:
# loader = TextLoader(r'D:\Amey Project\Chatbot\ml.txt')
path = r"D:\Amey Project\Chatbot\Data Services Page, Biz-Tech Analytics_Final Draft.docx"
loader = Docx2txtLoader(path)
basename = os.path.basename(path)
docs = loader.load() 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


### Store the knowledge base in ChromaDB.

In [26]:
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass()

vectorstore = Chroma.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever = vectorstore.as_retriever()

## Define the prompt.

In [27]:
# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use two sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [29]:
response = rag_chain.invoke({"input": "What is my name?"})
response["answer"]

"I don't know."

### Build the frontend UI using Gradio.

In [30]:
def predict(message, history):
    history_langchain_format = []
    prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    response = rag_chain.invoke({"input": f"{message}"})
    return response["answer"]



In [34]:
import gradio as gr

gr.ChatInterface(
    predict,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Type your query here", container=False, scale=7),
    title="Conversational RAG Chatbot",
    description=f"This chatbot will answer all queries related to {basename}.",
    theme="soft",
    examples=["What is machine learning?", "What is Data mining?", "State some applications for Machine Learning Algorithms."],
    cache_examples=True,
    retry_btn="Retry",
    undo_btn="Undo",
    clear_btn="Clear",
).launch(share=True)

Caching examples at: 'd:\Amey Project\Chatbot\gradio_cached_examples\73'
Caching example 1/3
Caching example 2/3
Caching example 3/3
Running on local URL:  http://127.0.0.1:7872
Running on public URL: https://b67f2777840970810f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
